In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords_nltk = set(stopwords.words('english'))
#stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 
                   #'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see',
                   #'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 
                   #'right', 'line', 'even', 'also', 'may', 'take', 'come'])

[nltk_data] Downloading package stopwords to /Users/lisun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv')
df

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5
20487,great location price view hotel great quick pl...,4
20488,"ok just looks nice modern outside, desk staff ...",2
20489,hotel theft ruined vacation hotel opened sept ...,1


In [3]:
#will use only the first 1,000 rows
df_1000 = df[0:1000]

In [5]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = df_1000.Review.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:5])

NameError: name 're' is not defined

In [57]:
vectorizer = CountVectorizer(stop_words = 'english')
doc_term = vectorizer.fit_transform(df_1000.Review)

doc_term.shape

(1000, 8812)

In [59]:
doc_term_df = pd.DataFrame(doc_term.toarray(), index=df_1000, columns=vectorizer.get_feature_names())
doc_term_df

,00,000,00am,00for,00p,00pm,03,04,05,05i,...,zero,zillion,zip,zipped,zipping,zoe,zone,zoo,äcor,ærom
"(nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night, , 4)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle, start booked suite paid extra website description not, suite bedroom bathroom standard hotel room, took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast, got kidding, embassy suits sitting room bathroom bedroom unlike kimpton calls suite, 5 day stay offer correct false advertising, send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution, send email trip guest survey did not follow email mail, guess tell concerned guest.the staff ranged indifferent not helpful, asked desk good breakfast spots neighborhood hood told no hotels, gee best breakfast spots seattle 1/2 block away convenient hotel does not know exist, arrived late night 11 pm inside run bellman busy chating cell phone help bags.prior arrival emailed hotel inform 20th anniversary half really picky wanted make sure good, got nice email saying like deliver bottle champagne chocolate covered strawberries room arrival celebrate, told needed foam pillows, arrival no champagne strawberries no foam pillows great room view alley high rise building good not better housekeeping staff cleaner room property, impressed left morning shopping room got short trips 2 hours, beds comfortable.not good ac-heat control 4 x 4 inch screen bring green shine directly eyes night, light sensitive tape controls.this not 4 start hotel clean business hotel super high rates, better chain hotels seattle, , 2)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(nice rooms not 4* experience hotel monaco seattle good hotel n't 4* level.positives large bathroom mediterranean suite comfortable bed pillowsattentive housekeeping staffnegatives ac unit malfunctioned stay desk disorganized, missed 3 separate wakeup calls, concierge busy hard touch, did n't provide guidance special requests.tv hard use ipod sound dock suite non functioning. decided book mediterranean suite 3 night weekend stay 1st choice rest party filled, comparison w spent 45 night larger square footage room great soaking tub whirlpool jets nice shower.before stay hotel arrange car service price 53 tip reasonable driver waiting arrival.checkin easy downside room picked 2 person jacuzi tub no bath accessories salts bubble bath did n't stay, night got 12/1a checked voucher bottle champagne nice gesture fish waiting room, impression room huge open space felt room big, tv far away bed chore change channel, ipod dock broken disappointing.in morning way asked desk check thermostat said 65f 74 2 degrees warm try cover face night bright blue light kept, got room night no, 1st drop desk, called maintainence came look thermostat told play settings happy digital box wo n't work, asked wakeup 10am morning did n't happen, called later 6pm nap wakeup forgot, 10am wakeup morning yep forgotten.the bathroom facilities great room surprised room sold whirlpool bath tub n't bath amenities, great relax water jets going, , 3)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(unique, great stay, wonderful time h

In [60]:
nmf = NMF(5, init = "nndsvda")
nmf.fit(doc_term)
nmf

NMF(init='nndsvda', n_components=5)

In [61]:
topic_term = nmf.components_.round(3)
topic_term.shape

(5, 8812)

In [62]:
topic_term_df = pd.DataFrame(topic_term.round(3),
                index = ["component_1", "component_2", "component_3", "component_4", "component_5"],
                columns = vectorizer.get_feature_names())
topic_term_df

,00,000,00am,00for,00p,00pm,03,04,05,05i,...,zero,zillion,zip,zipped,zipping,zoe,zone,zoo,äcor,ærom
component_1,0.105,0.013,0.000,0.007,0.004,0.020,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.006,0.000,0.000
component_2,0.091,0.002,0.036,0.002,0.000,0.012,0.000,0.000,0.009,0.015,...,0.019,0.000,0.001,0.00,0.003,0.000,0.026,0.000,0.009,0.000
component_3,0.055,0.000,0.001,0.000,0.000,0.006,0.007,0.027,0.019,0.000,...,0.000,0.002,0.000,0.00,0.001,0.009,0.010,0.009,0.000,0.004
component_4,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.018,0.000,0.011,0.00,0.000,0.000,0.000,0.000,0.000,0.000
component_5,0.099,0.003,0.015,0.000,0.005,0.000,0.005,0.000,0.006,0.000,...,0.012,0.008,0.000,0.01,0.000,0.014,0.030,0.003,0.038,0.000


In [63]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [64]:
output = display_topics(nmf, vectorizer.get_feature_names(), 25)
### END SOLUTION
output;


Topic  1
hotel, service, rooms, stay, staff, just, time, car, hotels, experience, stayed, area, like, lobby, restaurant, small, airport, recommend, did, parking, make, park, staying, bar, located

Topic  2
room, did, desk, night, stay, bathroom, bed, like, time, day, just, check, asked, staff, told, service, door, said, stayed, got, water, morning, shower, floor, new

Topic  3
great, location, staff, stay, room, stayed, friendly, service, just, place, really, city, helpful, little, walk, clean, excellent, price, comfortable, restaurants, walking, day, right, definitely, small

Topic  4
resort, did, good, staff, food, beach, just, water, evening, day, service, room, buffet, bar, kids, like, pool, shop, went, dominican, area, lobby, bus, tour, steakhouse

Topic  5
nice, good, rooms, seattle, place, stay, night, location, downtown, parking, clean, little, free, street, small, floor, bed, center, breakfast, friendly, walk, market, area, staff, blocks


